In [10]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import seaborn as sns

# Configure GPU memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define paths
BASE_DIR = '.'
DATASET_PATH = os.path.join(BASE_DIR, 'archive', 'DATASET')
TEST_LABELS_PATH = os.path.join(BASE_DIR, 'archive', 'test_labels.csv')
AUGMENTED_DATASET_PATH = os.path.join(BASE_DIR, 'Augmented_Dataset')
os.makedirs(AUGMENTED_DATASET_PATH, exist_ok=True)

# Constants
IMG_SIZE = 48
BATCH_SIZE = 32
NUM_CLASSES = 7

# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    try:
        img = load_img(image_path, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE))
        img_array = img_to_array(img) / 255.0  # Normalize
        return img_array
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

# Prepare dataset
def prepare_dataset():
    df = pd.read_csv(TEST_LABELS_PATH)
    images, labels = [], []
    
    for idx, row in df.iterrows():
        img_path = os.path.join(DATASET_PATH, 'test', str(row['label']), row['image'])
        img_array = load_and_preprocess_image(img_path)
        if img_array is not None:
            images.append(img_array)
            labels.append(tf.keras.utils.to_categorical(row['label'] - 1, NUM_CLASSES))
    
    return np.array(images), np.array(labels)

X, y = prepare_dataset()

# Define GAN
latent_dim = 100

def build_generator():
    model = keras.Sequential([
        layers.Dense(128 * 12 * 12, activation='relu', input_dim=latent_dim),
        layers.Reshape((12, 12, 128)),
        layers.Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu'),
        layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')
    ])
    return model

def build_discriminator():
    model = keras.Sequential([
        layers.Conv2D(64, (3, 3), strides=2, input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# GAN Training
def train_gan(epochs=10000, batch_size=32):
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        
        real_images = X[np.random.randint(0, X.shape[0], batch_size)]
        labels_real = np.ones((batch_size, 1))
        labels_fake = np.zeros((batch_size, 1))
        
        d_loss_real = discriminator.train_on_batch(real_images, labels_real)
        d_loss_fake = discriminator.train_on_batch(generated_images, labels_fake)
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, labels_real)
        discriminator.trainable = True
        
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss_real[0]}, G Loss: {g_loss}")

# Compile GAN
discriminator.trainable = False
z = keras.Input(shape=(latent_dim,))
generated_img = generator(z)
prediction = discriminator(generated_img)
gan = keras.Model(z, prediction)
gan.compile(optimizer='adam', loss='binary_crossentropy')

train_gan()

# Generate new images
num_new_images = 1000
noise = np.random.normal(0, 1, (num_new_images, latent_dim))
generated_images = generator.predict(noise)
np.save(os.path.join(AUGMENTED_DATASET_PATH, 'augmented_images.npy'), generated_images)

print("GAN-based augmentation complete!")

1/1 [==============================] - 0s 157ms/step
Epoch 0, D Loss: 0.6552926301956177, G Loss: 0.30527785420417786
1/1 [==============================] - 0s 48ms/step
Epoch 1000, D Loss: 0.4905722141265869, G Loss: 1.1900874376296997
1/1 [==============================] - 0s 37ms/step
Epoch 2000, D Loss: 1.5872361660003662, G Loss: 0.45200520753860474
1/1 [==============================] - 0s 44ms/step
Epoch 3000, D Loss: 0.4598709046840668, G Loss: 1.207617163658142
1/1 [==============================] - 0s 37ms/step
Epoch 4000, D Loss: 0.7473331689834595, G Loss: 1.2308632135391235
1/1 [==============================] - 0s 45ms/step
Epoch 5000, D Loss: 0.7933399677276611, G Loss: 1.0614516735076904
1/1 [==============================] - 0s 47ms/step
Epoch 6000, D Loss: 0.5864464640617371, G Loss: 0.6920320987701416
1/1 [==============================] - 0s 41ms/step
Epoch 7000, D Loss: 0.3664833605289459, G Loss: 1.1675188541412354
1/1 [==============================] - 0s 39ms/st

In [11]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, Model

# Define paths
BASE_DIR = '.'
AUGMENTED_DATASET_PATH = os.path.join(BASE_DIR, 'Augmented_Dataset', 'augmented_images.npy')
MODEL_PATH = os.path.join(BASE_DIR, 'ferplus_model.h5')
FINETUNED_MODEL_PATH = os.path.join(BASE_DIR, 'Fine-tuned model', 'fairness_aware_model.h5')
os.makedirs(os.path.dirname(FINETUNED_MODEL_PATH), exist_ok=True)

# Constants
BATCH_SIZE = 32
NUM_CLASSES = 7

# Load the augmented dataset
X_augmented = np.load(AUGMENTED_DATASET_PATH)
y_augmented = np.random.randint(0, NUM_CLASSES, size=(len(X_augmented),))
y_augmented = tf.keras.utils.to_categorical(y_augmented, NUM_CLASSES)

# Split into training and validation sets
split_idx = int(0.8 * len(X_augmented))
X_train, X_val = X_augmented[:split_idx], X_augmented[split_idx:]
y_train, y_val = y_augmented[:split_idx], y_augmented[split_idx:]

# Load Pretrained Model
base_model = load_model(MODEL_PATH)

# Modify the Model - Ensure Flattening
features = base_model.layers[-2].output
features = layers.Flatten()(features)

# Add New Output Layer
emotion_output = layers.Dense(NUM_CLASSES, activation="softmax", name="emotion_output")(features)

# Create New Model
fair_model = Model(inputs=base_model.input, outputs=emotion_output)

# Compile the Model
fair_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the Model
fair_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=BATCH_SIZE
)

# Save Fine-Tuned Model
fair_model.save(FINETUNED_MODEL_PATH)
print(f"Fine-tuned model saved at {FINETUNED_MODEL_PATH}")


Epoch 1/10
25/25 [==============================] - 7s 144ms/step - loss: 7.2389 - accuracy: 0.1238 - val_loss: 4.1360 - val_accuracy: 0.1150
Epoch 2/10
25/25 [==============================] - 3s 128ms/step - loss: 4.2454 - accuracy: 0.1663 - val_loss: 3.7109 - val_accuracy: 0.1100
Epoch 3/10
25/25 [==============================] - 3s 126ms/step - loss: 3.4289 - accuracy: 0.1538 - val_loss: 3.0522 - val_accuracy: 0.1250
Epoch 4/10
25/25 [==============================] - 3s 128ms/step - loss: 2.8082 - accuracy: 0.1988 - val_loss: 2.6415 - val_accuracy: 0.1000
Epoch 5/10
25/25 [==============================] - 3s 124ms/step - loss: 2.5261 - accuracy: 0.1887 - val_loss: 2.4950 - val_accuracy: 0.0900
Epoch 6/10
25/25 [==============================] - 4s 146ms/step - loss: 2.2869 - accuracy: 0.2075 - val_loss: 2.4178 - val_accuracy: 0.0900
Epoch 7/10
25/25 [==============================] - 3s 139ms/step - loss: 2.1892 - accuracy: 0.2150 - val_loss: 2.3767 - val_accuracy: 0.1300
Epoch 

c:\Users\Alyssa Vivien\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
